In [2]:
from glob import glob
import os
import fitz  # PyMuPDF 라이브러리
import pdfplumber
import sys
import re
import pandas as pd
from tqdm import tqdm
import traceback


c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\choi seung un\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def find_sentences(text, start_words, end_words=['\n']):
    for start_word in start_words:
        for end_word in end_words:
            pattern = re.compile(f'{start_word}.*?\{end_word}', re.DOTALL)
            matches = pattern.findall(text)
            if len(matches) > 0:
                break
        if len(matches) > 0:
            break
    result = matches[0].replace(start_word, "")
    result = result.replace(end_word, "")
    return result

### use fitz
def read_pdf_use_fitz(file_path, page_number=0):
    doc = fitz.open(file_path)
    page = doc[page_number]
    texts = page.get_text()
    prod_name = find_sentences(texts, ["상품명:", "상품명 :", "상 품 명", "상품명"])
    prod_feat = find_sentences(texts, ["상품특징", "특징"], ["거래조건", "거래 조건"])
    prod_name = prod_name.replace(":", "")
    prod_feat = prod_feat.replace(":", "")
    prod_feat = prod_feat.replace("\n", " ")
    prod_feat = prod_feat.replace("2", "")
    doc.close()
    return prod_name.strip(), prod_feat.strip()

def pdf_to_dataframe(pdf_path):
    page_number = 0
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_number]
        table = list()
        for i in page.extract_tables()[0][2:]:
            table.append([j for j in i if j not in ["", None]] )
        table = [i for i in table if len(i) != 0]
        # print("page_number", page_number+1, len(table))
        if len(table) < 2:
            table = list()
            for i in page.extract_tables()[3:][0]:
                table.append([j for j in i if j not in ["", None]] )
            table = [i for i in table if len(i) != 0]
            # print("page_number", page_number+1, len(table))
            
        if len(table) < 2:
            table = list()
            for i in page.extract_tables()[4:][0]:
                table.append([j for j in i if j not in ["", None]] )
            table = [i for i in table if len(i) != 0]
            # print("page_number", page_number+1, len(table))

        table = [[i[0], " ".join(i[1:])] if len(i) > 2 else i for i in table]
        table = [[None, i[0]] if len(i) == 1 else i for i in table]
        df = pd.DataFrame(table)
        # 첫 번째 행을 컬럼으로 사용
        df.columns = df.iloc[0]
        # 첫 번째 행은 더 이상 필요 없으므로 제거
        df = df[1:]
        columns = df.columns
        page_number += 1

        while True:
            page = pdf.pages[page_number]
            table_1 = list()    
            for i in page.extract_tables()[0]:
                table_1.append([j for j in i if j not in ["", None]] )
            table_1 = [i for i in table_1 if len(i) != 0]
            table_1 = [[i[0], " ".join(i[1:])] if len(i) > 2 else i for i in table_1]
            table_1 = [[None, i[0]] if len(i) == 1 else i for i in table_1]
            
            # print("page_number", page_number+1, len(table_1))
            
            if len(table_1) < 2:
                if len(page.extract_tables()) > 1:
                    table_1 = list()
                    for i in page.extract_tables()[1]:
                        table_1.append([j for j in i if j not in ["", None]] )
                    table_1 = [i for i in table_1 if len(i) != 0]
                    table_1 = [[i[0], " ".join(i[1:])] if len(i) > 2 else i for i in table_1]
                    table_1 = [[None, i[0]] if len(i) == 1 else i for i in table_1]
                    # print("page_number", page_number+1, len(table_1))


            if len(table_1) < 2:
                table_1 = list()    
                for i in page.extract_tables()[-1]:
                    table_1.append([j for j in i if j not in ["", None]] )
                table_1 = [i for i in table_1 if len(i) != 0]
                table_1 = [[i[0], " ".join(i[1:])] if len(i) > 2 else i for i in table_1]
                table_1 = [[None, i[0]] if len(i) == 1 else i for i in table_1]
                # print("page_number", page_number+1, len(table_1))
            if len(table_1) > 0:
                etc = pd.DataFrame(table_1, columns=columns)
                if etc.at[0, columns[1]] == None:
                    etc.at[0, columns[1]] = etc.at[0, columns[0]]
                    etc.at[0, columns[0]] = None
                df = pd.concat([df, etc], axis=0)
            page_number += 1
            if page_number >= len(pdf.pages):
                # print(f"End of the document. Processed {page_number} pages.")
                break
        
        df.reset_index(drop=True, inplace=True)
        ### 중간 구분에 괄호로만 있는 행이 있어서
        df[columns[0]] = df[columns[0]].str.replace("\n", "")
        df[columns[0]] = df[columns[0]].str.replace(" ", "")
        df[columns[0]] = df[columns[0]].astype(str).apply(lambda x: re.sub(r'\(.*', '', x))
        pattern = re.compile(re.escape("혜택") + '.*?' + re.escape("."))
        df[columns[0]] = df[columns[0]].astype(str).apply(lambda x: re.sub(pattern, '', x))


        # null인 행들을 순회하면서 이전 행의 '내용'에 추가하고 해당 행 삭제
        null_rows = df[(df[columns[0]].isin(["", "None"])) | (df[columns[0]].isna())]
        for index, row in null_rows.iterrows():
            prev_index = index - 1
            if prev_index >= 0:
                while True:
                    try:
                        df.at[prev_index, columns[1]] += f"\n{row[columns[1]]}"
                        break
                    except KeyError:
                        prev_index -= 1
                        continue
            df.drop(index, inplace=True)
        df.reset_index(drop=True, inplace=True)


        # null인 행들을 순회하면서 이전 행의 '구분'에 추가하고 해당 행 삭제
        null_rows = df[(df[columns[1]].isin(["", "None"])) | (df[columns[1]].isna())]
        
        for index, row in null_rows.iterrows():
            prev_index = index - 1
            if prev_index >= 0:
                while True:
                    try:
                        df.at[prev_index, columns[0]] += f"\n{row[columns[0]]}"
                        break
                    except KeyError:
                        prev_index -= 1
                        continue
            df.drop(index, inplace=True)
        df.reset_index(drop=True, inplace=True)


        def change_spec(text):
            try:
                text = text.strip()
                if text.startswith("-"):
                    text = text.replace("-", "●")
            except AttributeError:
                pass
            return text
        df[columns[0]] = df[columns[0]].apply(lambda x: x.strip()[1:] if x.strip().startswith("-") else x.strip())
        df[columns[1]] = df[columns[1]].apply(lambda x: change_spec(x))
        df.reset_index(drop=True, inplace=True)

        ## 구분에 중복되는 것은 병합해서 1개의 행으로 만듦
        df = df.groupby(df.columns[0])[df.columns[1]].agg(lambda x: '\n'.join(x)).reset_index()
        
        df.drop(df[df[columns[0]].isin(["0.1", "구분"])].index, inplace=True, axis=0)
        df.reset_index(drop=True, inplace=True)
        
    return df


In [8]:
pdf_path = "../data/은행상품/국민은행/KB두근두근여행적금.pdf"
# page_number = 0
print(read_pdf_use_fitz(pdf_path))
pdf_to_dataframe(pdf_path)


('KB 두근두근여행적금', '여행준비의 혜택과 설렘이 있는 적금  ❶ 친구들과 함께 받는 여행친구 우대이율 등 6개월제 최고이율 연 3.90%  ❷ 노랑풍선 여행상품 최대 3 만원 & 4% 할인 쿠폰번호 제공')


,구분,내용
0,가입금액및저축방법,월 5만원 이상 100만원 이하 (만원 단위)\n※ 신규 시 약정한 저축금액을 매월...
1,가입대상,"실명의 개인 (1인 1계좌)\n※ 개인사업자, 임의단체 및 공동명의 가입 불가(서류..."
2,가입방법,KB스타뱅킹
3,계약기간,6개월
4,계약해지방법,"• KB스타뱅킹, 인터넷뱅킹, 영업점 방문, 고객센터\n※ 고객센터 해지 시 만기해..."
5,기본이율,연 2.90%\n※ 기본이율: 신규가입일 당시 영업점 및 KB국민은행 홈페이지에 게...
6,만기앞당김지급,불가능
7,만기이자계산방법,{월저축금 × 계약월수 × (계약월수+1)/2} × (연이율/12)
8,만기후이율,신규가입일 당시 영업점 및 KB국민은행 홈페이지에 게시한 경과기간별\n만기후이율 적...
9,상품유형,정액적립식 예금


In [7]:
df = pd.DataFrame()
paths = "../data/은행상품/국민은행/*.pdf"
error_log_path = "./error.txt"
error_f = open(error_log_path, "w", encoding="utf-8-sig")

for i, path in enumerate(tqdm(glob(paths))):
# for i, path in enumerate(glob(paths)):
    
    # print("start", os.path.basename(path))
    try:
        prod_name, prod_feat = read_pdf_use_fitz(path)
        etc = pdf_to_dataframe(path)
        prod_name_index = len(etc)+1
        etc.at[prod_name_index, etc.columns[0]] = "상품명"
        etc.at[prod_name_index, etc.columns[1]] = prod_name
        prod_feat_index = len(etc)+1
        etc.at[prod_feat_index, etc.columns[0]] = "상품특징"
        etc.at[prod_feat_index, etc.columns[1]] = prod_feat
        etc.set_index(etc.columns[0], inplace=True)
        etc = etc.T.reset_index(drop=True)
        error_f.write(os.path.basename(path)+"\n")
        error_f.write(", ".join(etc.columns)+"\n")
        
        df = pd.concat([df, etc], axis=0)
        df.reset_index(drop=True, inplace=True)
        
    except Exception as e:
        error_f.write(os.path.basename(path)+"\n")
        traceback.print_exc(file=error_f)  # traceback를 파일에 직접 기록
        pass
error_f.close()

# 변경하고자 하는 특정 컬럼의 이름 설정
target_column = ["상품명", "상품특징"]

# 특정 컬럼의 위치를 변경
columns_except_target = [col for col in df.columns if col not in target_column]
target_column.extend(columns_except_target)
df = df[target_column]

path = "../data/은행상품/국민.csv"
df.to_csv(path, encoding="utf-8-sig")

100%|██████████| 58/58 [01:23<00:00,  1.44s/it]


In [78]:
###### 후처리

path = "../data/은행상품/국민_backup.csv"
df = pd.read_csv(path, encoding="utf-8-sig", index_col=0)

# df['가입금액'] = df['가입금액'].fillna('') + df['가입금액및저축기간'].fillna('')
# df['가입금액'] = df['가입금액'].fillna('') + df['가입금액및저축방법'].fillna('')
# df.drop(['가입금액및저축기간', '가입금액및저축방법'], axis=1, inplace=True)

# df['가입기간'] = df['가입기간'].fillna('') + df['가입신청및가입일정'].fillna('')
# df.drop(['가입신청및가입일정'], axis=1, inplace=True)

# df['가입자격'] = df['가입자격'].fillna('') + df['가입제한'].fillna('')
# df.drop(['가입제한'], axis=1, inplace=True)

# df['거래제한'] = df['거래제한'].fillna('') + df['거래제한직접'].fillna('')
# df.drop(['거래제한직접'], axis=1, inplace=True)

# df['기본이율'] = df['기본이율'].fillna('') + df['기본이율및이자계산방법'].fillna('')
# df.drop(['기본이율및이자계산방법'], axis=1, inplace=True)

# df['수수료면제'] = df['수수료면제'].fillna('') + df['수수료면제서비스'].fillna('')
# df['수수료면제'] = df['수수료면제'].fillna('') + df['수수료면제혜택'].fillna('')
# df['수수료면제'] = df['수수료면제'].fillna('') + df['전자금융/자동화기기수수료면제'].fillna('')
# df['수수료면제'] = df['수수료면제'].fillna('') + df['기본수수료면제서비스'].fillna('')
# df.drop(['수수료면제서비스', '수수료면제혜택', '전자금융/자동화기기수수료면제', '기본수수료면제서비스'], axis=1, inplace=True)

# df['양도및담보제공'] = df['양도및담보제공'].fillna('') + df['양도'].fillna('')
# df.drop(['양도'], axis=1, inplace=True)

# df['만기후이율'] = df['만기후이율'].fillna('') + df['만기후이율계산방법'].fillna('')
# df.drop(['만기후이율계산방법'], axis=1, inplace=True)

# df['연계·제휴서비스제공조건'] = df['연계·제휴서비스제공조건'].fillna('') + df['연계·제휴서비스제공조건및사용방법'].fillna('')
# df.drop(['연계·제휴서비스제공조건및사용방법'], axis=1, inplace=True)

# df['연계·제휴서비스내용'] = df['연계·제휴서비스내용'].fillna('') + df['연계ㆍ제휴서비스'].fillna('')
# df.drop(['연계ㆍ제휴서비스'], axis=1, inplace=True)

# df['이자계산방법'] = df['이자계산방법'].fillna('') + df['이자계산'].fillna('')
# df.drop(['이자계산'], axis=1, inplace=True)

# df['휴면예금및출연'] = df['휴면예금및출연'].fillna('') + df['휴면예금출연'].fillna('')
# df.drop(['휴면예금출연'], axis=1, inplace=True)

# df['환율우대서비스'] = df['환율우대서비스'].fillna('') + df['환율우대혜택'].fillna('')
# df.drop(['환율우대혜택'], axis=1, inplace=True)


df['예금자보호여부'] = df['예금자보호여부'].fillna('') + df['예금자보호'].fillna('')
df.drop(['예금자보호'], axis=1, inplace=True)

sorted(df.columns)

['가입1년경과후자동전환',
 '가입금액',
 '가입기간',
 '가입대상',
 '가입자격',
 '가입채널',
 '개인소득구간',
 '거래방법',
 '거래제한',
 '거래중지',
 '거래해지방법',
 '계약기간',
 '계약해지방법',
 '기본이율',
 '기타안내사항',
 '내용',
 '담보위험',
 '담보제공',
 '만기앞당김지급',
 '만기예금편리입금서비스',
 '만기이율',
 '만기이자계산방법',
 '만기일변경',
 '만기후이율',
 '명의변경',
 '변경·종료에대한사전통지',
 '부가서비스',
 '분할해지및중도해지이율계산방법',
 '비상금이율적용조건',
 '상품명',
 '상품유형',
 '상품특징',
 '서비스항목',
 '세제관련',
 '세제혜택',
 '수수료면제',
 '신규금액',
 '알아두실사항',
 '양도및담보제공',
 '연계·제휴서비스내용',
 '연계·제휴서비스변경·종료에대한사전통지',
 '연계·제휴서비스이행책임',
 '연계·제휴서비스제공기간',
 '연계·제휴서비스제공조건',
 '예금의일부인출',
 '예금이율우대',
 '예금자보호여부',
 '외환수수료우대',
 '우대서비스',
 '우대이율',
 '원금및이자지급제한',
 '월균등분할원금변경',
 '위법계약해지권',
 '유의사항',
 '이용제한사항',
 '이자계산방법',
 '이자및원금지급시기',
 '이자지급방법',
 '이자지급시기',
 '이행책임',
 '일부인출',
 '일부해지',
 '입금한도',
 '자동이체',
 '자동전환',
 '자료열람요구권',
 '재가입및조정비율적용',
 '재예치',
 '저금통기능',
 '저축금액',
 '저축방법및저축금액',
 '적용이율',
 '전환가능여부',
 '접속경로',
 '정부기여금',
 '제공기간',
 '제공대상',
 '제공조건',
 '중도해지이율',
 '중도해지이자계산방법',
 '지급금액',
 '지급대상',
 '질권설정',
 '총급여기준',
 '최종이율',
 '추가입금',
 '쿠폰금액',
 '통장발행',
 '특별수수료면제서비스',
 '특별이율',
 '특별중도해지',
 '해

In [79]:
path = "../data/은행상품/국민_backup.csv"
df.to_csv(path, encoding="utf-8-sig")

In [80]:
# filtered_df = df[[col for col in df.columns if col in ["만기이율", "만기후이율"]]]
filtered_df = df[[col for col in df.columns if "예금자보호" in col]]
rows_with_values = filtered_df[filtered_df.apply(lambda row: row.count() >= 2, axis=1)]
print(len(rows_with_values))
filtered_df

0


,예금자보호여부
0,"해당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는 본\n은행..."
1,"해 당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는 본\n은..."
2,"해당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는 본\n은행..."
3,"해 당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호\n한도는 본 은..."
4,"해 당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호\n한도는 본 은..."
5,"해 당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호\n한도는 본 은..."
6,"해 당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는\n본 은..."
7,"해 당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는\n본 은..."
8,"해당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는 본\n은행..."
9,"해당 이 예금은 예금자보호법에 따라 예금보험공사가 보호하되, 보호 한도는 본\n은행..."
